In [ ]:
import requests,time,random
from lxml import etree
from urllib import parse
from selenium import webdriver 
import pandas as pd 
import numpy as np
import time
from selenium.webdriver.common.by import By
import random
import re
import numpy as np
import pandas as pd
from chemdataextractor import Document
from chemdataextractor.model import Compound
from chemdataextractor.doc import Paragraph, Heading
from pymatgen.core.composition import Composition
import re
import copy
import traceback
import time

In [ ]:
## data and url preparation
ls= pd.read_csv('ORR_list.csv',header=None)
ls.dropna()
ls=np.array(ls)
full_list=ls.tolist()
url_base='https://www.webofscience.com/wos/alldb/basic-search'

In [ ]:
len(full_list)

In [ ]:
query_list=full_list[0:10000]

In [ ]:
# query_list[1746] we have 1747 records 
def split_list(listTemp, step):
    list_result=[]
    for i in range(0, len(listTemp), step):
        list_result.append(listTemp[i: i + step])
    return list_result

In [ ]:
splitted=split_list(query_list,20)

In [ ]:
len(query_list)

In [ ]:
#query_list
options = webdriver.chrome.options.Options()
# options.add_argument("--auto-open-devtools-for-tabs")

In [ ]:
###not needed keywords for material and elements###
Not_needed_key_word_list=[
'Pt','Pt/C-RuO2','RuO2-Pt/C','Pt/C-RuO2/C','RuO2/C-Pt/C',
'PBS','KOH','Pt/C','pt/c','Pt/C-Ru/C','Ru/C-Pt/C','OH',
'Ru/C','ru/c','NaCl','nacl','khco3','KHCO3','NaOH',
'koh','naoh','H2SO4','HClO4','HCLO4','h2so4','hclo4'
]

In [ ]:
Keyword_pattern_HWP=re.compile("E\s?\-?\/?1\/2|half\s?\-?wave\s?\-?potential")
Acid_pattern=re.compile("[A|a]cid[ic]?|[H|h]2SO\(?4\)?|[H|h]ClO\(?4\)?|[S|s]ul[f|ph]uric\s?[A|a]cid|[P|p]erchloric\s?[A|a]cid|[P|p]erchlorate|pH 1")
Alkaline_pattern=re.compile("[A|a]lkali[ne|i|ic]|[K|k]OH|[N|n]aOH|[S|s]odium\s?[H|h]ydroxide|[P|p]ota[ssium|sh]\s?hydroxide|pH 13|pH 14")
Neutral_pattern=re.compile("[N|n]eutral|PBS|[P|p]?[hosphate]?\s?[B|b]uffer|[S|s]aline|pH 7")
###(Not HClO4)XX.XX mV|XX.XX, | XX.XX and#####
Value_Keyword_pattern=re.compile("\d+\s?V(?!\s?[D|d]ec)|\d+\.\d+\s?V(?!\s?[D|d]ec)|(?<![O|-])\d+\,\s?|(?<![O|-])\d+\.\d+\,\s?|(?<![O])\d+\sand|(?<![O])\d+\.\d+\sand")

In [ ]:
###Clean the marks .@/-x###
def clear_mark(target_list):
#     print(target_list)
    try:
        for i in range(0,len(target_list)):
            ###remove chemical value e.g. Ir(IV)###
            target_list[i]=re.sub('\(V?[I]+V?\)','',target_list[i])
            ###replacing special specifiers###
            target_list[i]=target_list[i].replace('/',"")
            target_list[i]=target_list[i].replace('@',"")
            target_list[i]=target_list[i].replace('-',"")
            target_list[i]=target_list[i].replace('x',"")
            target_list[i]=target_list[i].replace('y',"")
            target_list[i]=target_list[i].replace('X',"")
            
    except:
        pass
    return target_list

In [ ]:
def Compare_whether_same_element_exist(list_a,list_b):
    Judge=False
    for each_a in list_a:
        if each_a in list_b:
            Judge=Judge or True
        else:
            Judge=Judge or False
    return Judge

In [ ]:
def Check_and_supplement_element(key_word_list,Extracted,Element_list,Element):
    if Compare_whether_same_element_exist(key_word_list,Extracted)==True:
        Element_list.append(Element)
    return Element_list

In [ ]:
def standardize_element(Element_list,number_elements):
    return_list=copy.deepcopy(Element_list)
    if len(Element_list)<number_elements:
        for i in range(0,number_elements-len(Element_list)):
            return_list.append('None')
    else:
        return_list=return_list[0:number_elements]
    return return_list

In [ ]:
########Material Speciality##########
def Find_Material(Pattern,text):
    return_list=[]
    Judge=0
    Result=re.findall(Pattern,text,re.IGNORECASE)
    if len(Result)>=1:
        Judge=1
    return_list.append(Judge)
    return return_list

In [ ]:
###Find the key sentence from the text for further mining###
def Find_Target_Sentence_From_Text(text,Keyword_pattern):
    sentence_list=text.split('. ')
    HWP_found_sentence_list=[]
    for each_sentence in sentence_list:
        try:
            HWP_index_list=[m.start(0) for m in re.finditer(Keyword_pattern, each_sentence)]
            if len(HWP_index_list)>=1:
                HWP_found_sentence_list.append(each_sentence)
        except:
            print('error!')
    return HWP_found_sentence_list

In [ ]:
###Turn extracted voltage list like [' 15 mV', ' 512 mV'] into [15.0, 512.0]###
def extract_value(target_list):
#     print(target_list)
    try:
        for i in range(0,len(target_list)):
            target_list[i]=float(target_list[i].strip(' ,mVand'))
    except:
        pass
    return target_list

In [ ]:
#############Check_Irregular###############
def Check_Irregular(HWP_values_list,Acid_index_list,Alkaline_index_list,Neutral_index_list,Low_Value_Boundary,High_Value_Boundary,HWP_Multiple_Boundary,Acid_Keyword_Boundary,Alkaline_Keyword_Boundary,Neutral_Keyword_Boundary):
    ###Check if there is irregular high value like OER HWP in the same sentence### 
    Irregular_Abnormal_Value=0
    for each_founded_HWP_value in HWP_values_list:
        if each_founded_HWP_value>High_Value_Boundary or each_founded_HWP_value<Low_Value_Boundary:
#             print('Irregular value not in range found!, value is',each_founded_HWP_value)
            Irregular_Abnormal_Value=1
    ###Check if multi values found### 
    Irregular_Multiple_Value=0
    if len(HWP_values_list)>HWP_Multiple_Boundary:
       Irregular_Multiple_Value=1
    ###Check if multi keywords found### 
    Irregular_Multiple_Keywords=0
    if (len(Acid_index_list)>Acid_Keyword_Boundary)*(len(Alkaline_index_list)>Alkaline_Keyword_Boundary)*(len(Neutral_index_list)>Neutral_Keyword_Boundary)==1:
       Irregular_Multiple_Keywords=1
    return Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords

In [ ]:
####defining init database##
database=pd.DataFrame(columns=[
####publication informations####
'DOI','Country', 'Facility', 'Journal_Name', 'IF', 'Indexed_Date','Cited_Time',
####needed performance output####
'Acid_HWP','Alkaline_HWP','Neutral_HWP','Remark_Sentence','Bug_Note',
####records of irregular informations
'Irregular_Abnormal_Value','Irregular_Multiple_Value','Irregular_Multiple_Keywords',
####elemental informations####
'Element_1','Element_2','Element_3','Element_4','Element_5','Element_6','Element_7','Element_8',
####one-hot features####
'Judge_Nanoparticles','Judge_3D','Judge_2D','Judge_1D','Judge_Urchin','Judge_SAC','Judge_QD',
'Judge_Hollow','Judge_Core_Shell','Judge_Template','Judge_alloy','Judge_Functionalized',
'Judge_heterostructures','Judge_Polymer_Complex_Chelate','Judge_Biomass_Slag','Judge_MOF','Judge_COF',
'Judge_Conductive_Polymer','Judge_Polymetal','Judge_Hydroxide_Oxyhydroxide','Judge_oxalate','Judge_Spinel',
'Judge_Perovskite','Judge_Pyrochlore','Judge_Prussian_Blue','Judge_Amorphous','Judge_Ultrasound','Judge_Plasma',
'Judge_Graphdiyne','Judge_Graphene','Judge_c3n4','Judge_MXene','Judge_CNT','Judge_IL','Judge_Organic_Inorganic',
'Judge_Steel','Judge_Carbon_Cloth','Judge_Film','Judge_Molten','Judge_Co_Precipitation','Judge_Electrospin',
'Judge_Hydrothermal','Judge_Microwave','Judge_Spray','Judge_Phys_Chem_Deposition','Judge_Light_Deposition',
'Judge_Electrochemical_Deposition','Judge_Electrochemical_Corrosion','Judge_Etching','Judge_Sol_Gel',
####special warings####
'Warning_Onset'
]
)

In [ ]:
#############Recording###############
def Add_New_Record(
    DOI,Country,Facility,Journal_Name,IF,Indexed_Date,Cited_Time,
    Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,
    Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords,
    Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,
    Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,
    Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,
    Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,
    Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,
    Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,
    Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,
    Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,
    Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,
    Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset,
    database_previous):
    ###creating a new record###
    New_Record=pd.DataFrame(
    {'DOI':DOI,'Country':Country,'Facility':Facility,'Journal_Name':Journal_Name,
    'IF':IF,'Indexed_Date':Indexed_Date,'Cited_Time':Cited_Time,
    'Acid_HWP':Acid_HWP,'Alkaline_HWP':Alkaline_HWP,'Neutral_HWP':Neutral_HWP,
    'Remark_Sentence':Remark_Sentence,'Bug_Note':Bug_Note,
    'Irregular_Abnormal_Value':Irregular_Abnormal_Value,'Irregular_Multiple_Value':Irregular_Multiple_Value,
    'Irregular_Multiple_Keywords':Irregular_Multiple_Keywords,
     #####elemental information#####
    'Element_1':Element_1,'Element_2':Element_2,
    'Element_3':Element_3,'Element_4':Element_4,
    'Element_5':Element_5,'Element_6':Element_6,
    'Element_7':Element_7,'Element_8':Element_8,
     #####Judge Materials#####
    'Judge_Nanoparticles':Judge_Nanoparticles,'Judge_3D':Judge_3D,'Judge_2D':Judge_2D,'Judge_1D':Judge_1D,
    'Judge_Urchin':Judge_Urchin,'Judge_SAC':Judge_SAC,'Judge_QD':Judge_QD,'Judge_Hollow':Judge_Hollow,
    'Judge_Core_Shell':Judge_Core_Shell,'Judge_Template':Judge_Template,'Judge_alloy':Judge_alloy,
    'Judge_Functionalized':Judge_Functionalized,'Judge_heterostructures':Judge_heterostructures,
    'Judge_Polymer_Complex_Chelate':Judge_Polymer_Complex_Chelate,'Judge_Biomass_Slag':Judge_Biomass_Slag,
    'Judge_MOF':Judge_MOF,'Judge_COF':Judge_COF,'Judge_Conductive_Polymer':Judge_Conductive_Polymer,
    'Judge_Polymetal':Judge_Polymetal,'Judge_Hydroxide_Oxyhydroxide':Judge_Hydroxide_Oxyhydroxide,
    'Judge_oxalate':Judge_oxalate,'Judge_Spinel':Judge_Spinel,'Judge_Perovskite':Judge_Perovskite,
    'Judge_Pyrochlore':Judge_Pyrochlore,'Judge_Prussian_Blue':Judge_Prussian_Blue,'Judge_Amorphous':Judge_Amorphous,
    'Judge_Ultrasound':Judge_Ultrasound,'Judge_Plasma':Judge_Plasma,'Judge_Graphdiyne':Judge_Graphdiyne,
    'Judge_Graphene':Judge_Graphene,'Judge_c3n4':Judge_c3n4,'Judge_MXene':Judge_MXene,'Judge_CNT':Judge_CNT,
    'Judge_IL':Judge_IL,'Judge_Organic_Inorganic':Judge_Organic_Inorganic,'Judge_Steel':Judge_Steel,
    'Judge_Carbon_Cloth':Judge_Carbon_Cloth,'Judge_Film':Judge_Film,'Judge_Molten':Judge_Molten,
    'Judge_Co_Precipitation':Judge_Co_Precipitation,'Judge_Electrospin':Judge_Electrospin,
    'Judge_Hydrothermal':Judge_Hydrothermal,'Judge_Microwave':Judge_Microwave,'Judge_Spray':Judge_Spray,
    'Judge_Phys_Chem_Deposition':Judge_Phys_Chem_Deposition,'Judge_Light_Deposition':Judge_Light_Deposition,
    'Judge_Electrochemical_Deposition':Judge_Electrochemical_Deposition,
    'Judge_Electrochemical_Corrosion':Judge_Electrochemical_Corrosion,
    'Judge_Etching':Judge_Etching,'Judge_Sol_Gel':Judge_Sol_Gel,
    'Warning_Onset':Warning_Onset},
    index=[0])
    updated_database=database_previous.append(New_Record)
    return updated_database

In [ ]:
database

In [ ]:
def Extract_target_element_material(Not_needed_key_word_list,text):
    ###Parsing the text by chemdataextractor###
    d0 = Document(
        Paragraph(text),
        models = [Compound]
    )
    Parsed_Result=d0.records.serialize()
    ###Extracting Compound names###
    Parsed_Name_List=[]
    for each_dict in Parsed_Result:
#         print(each_dict)
        if 'names' in each_dict['Compound']:
            Parsed_Name_List.append(each_dict['Compound']['names'][0])
    ###removing some not needed names###
    ###Try to remove the not need chemical names###
    if Compare_whether_same_element_exist(Not_needed_key_word_list,Parsed_Name_List)==True:
        for each_not_neede in Not_needed_key_word_list:
            try:
                Parsed_Name_List.remove(each_not_neede)
            except:
                pass
    ###clear special characters###
    Extracted=clear_mark(Parsed_Name_List)
    ###analyze the chemical elements by pymatgen###
    Element_list=[]
    for each_compound_record in Extracted:
        try:
    #         print(each_compound_record)
            comp = Composition(each_compound_record)
    #         print(comp.elements)
            for each_element in comp.elements:
                Element_list.append(each_element.name)
        except:
            pass
    ###In case missing elements, supplementation for it###
    carbon_keyword_list=['graphitic','Graphitic','Carbon','carbon','Graphene','graphene','Graphite','graphite'
                        ,'Graphenes','graphenes','Graphites','graphites',"Carbide",'carbide',
                        "Carbides",'carbides','polyoxometalates-carbon black',
                        'carbonate','carbonates','Carbonates','Carbonate']
    oxide_keyword_list=['Oxide','oxide','Oxides','oxides']
    sulfur_keyword_list=['Sulfuric','sulfuric','Sulfur','sulfur','Sulfide','Sulphide',
                         'sulfide','sulphide','Sulfided','sulfided','Sulfurous','sulfurous',
                         'Sulfides','Sulphides','sulfides','sulphides','chalcogenide','chalcogenides'
                        'Chalcogenide','Chalcogenides']
    nitrogen_keyword_list=['Nitrogen','nitrogen','Nitride','nitride','Nitrides','nitrides']
    phosphorus_keyword_list=["Phosphine","phosphine","Phosphines","phosphines","Phosphorus",
                             "phosphorus","Phosphide","phosphide","Phosphides","phosphides",
                             "Phosphate","phosphate","Phosphates","phosphates","Phosphor",
                             "phosphor","Phosphors","phosphors"]
    boride_keyword_list=['Boride','boride','Borides','borides','Boron','Borons','boron',
                         'borons','Borate',"Borates",'borate','borates','Borite','Borites',
                        'borite','borites','Boricide','Boricides','boricide','boricides',
                        'Boriding','boriding','Boridings','boridings',"Boronide",'boronide']
    fluoride_keyword_list=['Fluoride','fluoride','Fluorides','fluorides','Fluorine','fluorine']                       
    chloride_keyword_list=['Chloride','chloride','Chlorides','chlorides',
                          'Chlorine','chlorine','Chlorines','chlorines','chalcogenide'
                          ,'chalcogenides',"Chalcogenide",'Chalcogenides']
    bromide_keyword_list=['Bromide',"bromide",'Bromides',"bromides",'Bromine',"bromine",
                          'Brominate',"brominate",'Brominated',"brominated",'Brominates',
                          'brominates','Bromines',"bromines",'Bromination','bromination'
                          ,'Brominations','brominations']
    selenide_keyword_list=['Selenide','selenide','Selenides','selenides',"Selenium",'selenium',
                           "Seleniums",'seleniums','Selenite','selenite','Selenites','selenites'
                           ,'Seldenide','seldenide','Seldenides','seldenides','Seelenide','seelenide'
                           ,'Seelenides','seelenides','Sederide','sederide','Sederides','sederides'
                           'dichalcogenide','dichalcogenides','Dichalcogenide','Dichalcogenides']
    telluride_keyword_list=['Telluride','telluride','Tellurium','tellurium',
                            'Tellurides','tellurides','Telluriums','telluriums',
                            'Telluronium','telluronium','Telluroniums','telluroniums',
                            'Teluride','teluride','Telurides','telurides','Telluriumide',
                            'Telluriumides','telluriumide','telluriumides','Tellurite',
                            'tellurite','Tellurites','tellurites']
    arsenic_keyword_list=['Arsenic','arsenic','Arsenical','arsenical','Asenic','asenic']
    silicide_keyword_list=['Silicide','silicide','Silicides','silicides','Siliconide','siliconide',
                           'Siliconides','siliconides','Silicaide','silicaide','Silicaides','silicaides'
                            ]
    ####new
    iodide_keyword_list=['iodide','iodides','iodide','iodides']
    ###supplement the new elements in the list###
    Element_list=Check_and_supplement_element(carbon_keyword_list,Extracted,Element_list,'C')
    Element_list=Check_and_supplement_element(oxide_keyword_list,Extracted,Element_list,'O')
    Element_list=Check_and_supplement_element(sulfur_keyword_list,Extracted,Element_list,'S')
    Element_list=Check_and_supplement_element(nitrogen_keyword_list,Extracted,Element_list,'N')
    Element_list=Check_and_supplement_element(phosphorus_keyword_list,Extracted,Element_list,'P')
    Element_list=Check_and_supplement_element(boride_keyword_list,Extracted,Element_list,'B')
    Element_list=Check_and_supplement_element(fluoride_keyword_list,Extracted,Element_list,'F')
    Element_list=Check_and_supplement_element(chloride_keyword_list,Extracted,Element_list,'Cl')
    Element_list=Check_and_supplement_element(bromide_keyword_list,Extracted,Element_list,'Br')
    Element_list=Check_and_supplement_element(selenide_keyword_list,Extracted,Element_list,'Se')
    Element_list=Check_and_supplement_element(telluride_keyword_list,Extracted,Element_list,'Te')
    Element_list=Check_and_supplement_element(arsenic_keyword_list,Extracted,Element_list,'As')
    Element_list=Check_and_supplement_element(silicide_keyword_list,Extracted,Element_list,'Si')
    Element_list=Check_and_supplement_element(iodide_keyword_list,Extracted,Element_list,'I')
    ###remove deprecated elements###
    Element_list=list(set(Element_list))
    ###remove H and sort,and finally standardize###
    Element_list.sort()
    try:
        Element_list.remove("H")
    except:
        print('No H in the element list.')
    final_element_list=standardize_element(Element_list,8)
    ###find keyword for material types
    ####nano scale####
    Judge_Nanoparticles=Find_Material('nano\s?\-?\/?(assembl|dot)|nano\s?\-?\/?polyhedra|bowl|micro\s?\-?\/?sphere|thorn|decahedr|nano\s?\-?\/?crystal[s]?|nano\s?\-?\/?particle[s]?|nano\s?\-?\/?sphere[s]?|nano\s?\-?\/?cluster[s]?|cluster[s]?|nano\s?\-?\/?prism[s]?|nano\s?\-?\/?cone[s]?|nano\s?\-?\/?triangle[s]?|nano\s?\-?\/?(net)?cage[s]?|nano\s?\-?\/?(cube|framework)[s]?|nano\s?\-?\/?bone[s]?|nano\s?\-?\/?dumbell[s]?|dumbell[s]?',text)
    Judge_3D=Find_Material('hydrogel|xerogel|aerogel|(nano|super)\s?\-?\/?structure|monolith|sponge|three\s?\-?\/?dimension(al)?|aerogel[s]?|foam|(?<![a-zA-Z])3\-?d|array|self\s?\-?\/?support(ing|ed)?|self\s?\-?\/?stand(ing|ed)?',text)
    Judge_2D=Find_Material('nano\s?\-?\/?lea|lamellar|flake|nano\s?\-?\/?flake|micro\s?\-?\/?sheet|two\s?\-?\/?dimension(al)?|nano\s?\-?\/?sheet[s]?|nano\s?\-?\/?plate[s]?|(?<![a-zA-Z])2\-?d|metallene',text)
    Judge_1D=Find_Material('tublar|coaxial|pillar|nano\s?\-?\/?wall|nano\s?\-?\/?worm[s]?|nano\s?\-?\/?chain[s]?|rod[s]?|one\s?\-?\/?dimension(al)?|(?<![a-zA-Z])1\-?d|nano\s?\-?\/?tube[s]?|nano\s?\-?\/?wire[s]?|nano\s?\-?\/?rod[s]?|nano\s?\-?\/?fiber[s]?|nano\s?\-?\/?needle[s]?|needle[s]?|nano\s?\-?\/?belt[s]?',text)
    Judge_Urchin=Find_Material('forest|dendritic|coral|(nano)?\s?\-?\/?dendrite|rosette|nano\s?\-?\/?coral|octopus|nano\s?\-?\/?branch|urchin[s]?|nano\s?\-?\/?flower[s]?|flower|nano\s?\-?\/?star[s]?',text)
    Judge_SAC=Find_Material('single\s?\-?\/?atom[s]?|single\s?\-?\/?site[s]?|atomical(ly)?\s?\-?\/?disperse[d]?|SAC[s]?',text)
    Judge_QD=Find_Material('(?<![a-zA-Z])0\-?d|quantum\s?dot[s]?|carbon\s?\-?\/?(nano)\s?\-?\/?dot[s]?|(?<![a-zA-Z])QD[s]?',text)
    Judge_Hollow=Find_Material('network|interconnect|meso\s?\-?\/?porous|hollow[s]?|nano\s?\-?\/?hollow|honey\s?\-?\/?comb|porous|channel',text)
    Judge_Core_Shell=Find_Material('onion|capsul|shell|encapsul|core\s?\-?\/?shell|yolk\s?\-?\/?shell',text)
    Judge_Template=Find_Material('(?!template free|template-free)(?:template)(?<!non-template|non template)(?<!without template)(?<!no need for template)',text)
    ####material####
    Judge_alloy=Find_Material('alloy|metallene|intermetal[lic]?\s?(compound)?',text)
    Judge_Functionalized=Find_Material('animate|functional(ized)]?|graft|modified',text)
    Judge_heterostructures=Find_Material('nano\s?\-?\/?composite|hierarchical|interface|junction|het\-?ero(structure)?|hybrid[s]?',text)
    Judge_Polymer_Complex_Chelate=Find_Material('Polyaniline|Polydopamine|polymer|complex|chelate',text)
    Judge_Biomass_Slag=Find_Material('bio\s?\-?inspire|natural|vitamin|hemin|bacterial\s?\-?\/cellulose|slag|biomass|biowaste|waste|biosource',text)
    Judge_MOF=Find_Material('metal\s?\-?\/?organic\s?\-?\/?framework|MOF|zeolit(e|ic)\s?\-?\/?imidazole\s?\-?\/?framework[s]?|ZIF',text)
    Judge_COF=Find_Material('Triazine\s?\-?\/?framework|Triazine\s?\-?\/?organic\s?\-?\/?framework|COF|covalent\s?\-?\/?organic\s?\-?\/?framework[s]?|organic\s?\-?\/?framework',text)
    Judge_Conductive_Polymer=Find_Material('Polyphenyl\s?\-?acetylene|Polyphenylene|Polyphenylamine|Polyaniline|Polythiophene|Polyacetylene|polypyrrole',text)                           
    Judge_Polymetal=Find_Material('polyoxo|poly\-?oxometalate[s]?|poly\-?metal(lic)?|(?<![a-zA-Z])POM[s]?',text)
    Judge_Hydroxide_Oxyhydroxide=Find_Material('hydroxide|LDH|oxyhydroxide',text)
    Judge_oxalate=Find_Material('oxalate',text)
    Judge_Spinel=Find_Material('spinel',text)
    Judge_Perovskite=Find_Material('perovskite',text)
    Judge_Pyrochlore=Find_Material('pyrochlore',text)
    Judge_Prussian_Blue=Find_Material('prussian\s?\-?blue',text)
    Judge_Amorphous=Find_Material('Amorphous|Amorphic',text)
    Judge_Ultrasound=Find_Material('ultrasonic|ultrasound',text)
    Judge_Plasma=Find_Material('plasma',text)                           
    Judge_Graphdiyne=Find_Material('graphdiyne|GDY',text)
    Judge_Graphene=Find_Material('graphene',text)
    Judge_c3n4=Find_Material('g\-?c\(?3\)?n\(?4\)?|g\-?carbon\s?\-\?nitride',text)
    Judge_MXene=Find_Material('mxene[s]?',text)
    Judge_CNT=Find_Material('cnt|carbon\s?\-?nano\s?\-?\/?tube[s]?|carbon\s?nano\s?\-?\/?(wire|fiber)[s]?',text)
    Judge_IL=Find_Material('(?<![a-zA-Z])IL(?![a-zA-Z])|ion(ic)?\s?\-?liquid',text)
    Judge_Organic_Inorganic=Find_Material('Organic\-?\s?\/?Inorganic',text)                        
    Judge_Steel=Find_Material('steel',text)
    Judge_Carbon_Cloth=Find_Material('carbon\s?\-?\/?(cloth|fabric)',text)
    Judge_Film=Find_Material('film',text)
    Judge_Molten=Find_Material('molten',text)
    Judge_Co_Precipitation=Find_Material('co\s?\-?\/?(precip|sediment|deposit)|impregnat|dipp(ed|ing)|immers(e|ion)',text)                        
    Judge_Electrospin=Find_Material('electrosp[i|u]n',text)
    Judge_Hydrothermal=Find_Material('polyol|hydrotherm|one\s?\-?pot',text)
    Judge_Microwave=Find_Material('microwave',text)
    Judge_Spray=Find_Material('spray',text)
    Judge_Phys_Chem_Deposition=Find_Material('MBE|layer\s?\-?deposition|ALD|pulsed\s?\-?laser|laser\s?\-?deposition|PLD|CVD|PVD|(physical|chemical)?\s?\-?\/?vapor\s?deposition|vapor\s?\-?\/?grow|sputter',text)
    Judge_Light_Deposition=Find_Material('(photo|light|photochemical)\s?\/\-?(deposit|fabricat|reduc|prepar|form)',text)
    Judge_Electrochemical_Deposition=Find_Material('(underpotential|electro|electrochemical|electrode|galvanic)\s?\-?\/?(deposit|fabricat|reduc|prepar|form)',text)
    Judge_Electrochemical_Corrosion=Find_Material('(underpotential|electro|electrochemical|electrode|galvanic)\s?\-?\/?(displace|corros|dealloy)',text)
    Judge_Etching=Find_Material('(?:corrosion|etch(ing|ed)?|leach|acid wash)(?<!electrochemical corrosion)(?<!electrochemical etch)(?<!galvanic corrosion)(?<!galvanic etch)(?<!electro-chemical corrosion)(?<!electro-chemical etch)',text)
    Judge_Sol_Gel=Find_Material('sol\-?\s?\/?gel',text)            
    ###whether there might be OER records that confuse the final results given by the crawler###
    Warning_Onset=Find_Material('E\s?\-?\/?onset|on\s?\-?set\s?\-?potential',text)
    ####8+50+1=59####
    Element_1=final_element_list[0]
    Element_2=final_element_list[1]
    Element_3=final_element_list[2]
    Element_4=final_element_list[3]
    Element_5=final_element_list[4]
    Element_6=final_element_list[5]
    Element_7=final_element_list[6]
    Element_8=final_element_list[7]
    ##############################
    return Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset

In [ ]:
def Extracting_target_values(Acid_pattern,Alkaline_pattern,Neutral_pattern,Value_Keyword_pattern,Target_Sentence,Low_Value_Boundary,High_Value_Boundary,DOI):
    Acid_index_list=[m.start(0) for m in re.finditer(Acid_pattern, Target_Sentence)]
    Alkaline_index_list=[m.start(0) for m in re.finditer(Alkaline_pattern, Target_Sentence)]
    Neutral_index_list=[m.start(0) for m in re.finditer(Neutral_pattern, Target_Sentence)]
    print('Mention of acid:',len(Acid_index_list),'times; alkaline:',len(Alkaline_index_list),'times; neutral:',len(Neutral_index_list),'times',)
    ###########################
    ###No pH keywords at all###
    ###########################
    if (len(Acid_index_list)==0)*(len(Alkaline_index_list)==0)*(len(Neutral_index_list)==0)==1:
        print('No appearance of pH key word in the key sentence.')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Even though some HWP times values could be found### 
            final_HWP_value_single=max(HWP_values_list)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
            Acid_index_list,Alkaline_index_list,
            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=1,
            Acid_Keyword_Boundary=0,Alkaline_Keyword_Boundary=0,Neutral_Keyword_Boundary=0)
            ###Recording###
            Bug_Note='No appearance of pH key word in the key sentence. But a '+final_HWP_value_single+'of HWP was found.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='No pH key word.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    ##############################       
    ###Only acid：1 and 1+ time###
    ##############################
    elif (len(Acid_index_list)>=1)*(len(Alkaline_index_list)==0)*(len(Neutral_index_list)==0)==1:
        print('Only acid, appearance: ',len(Acid_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Most time, for single value, choose the maximum if multiple are found### 
            final_HWP_value_single=max(HWP_values_list)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=1,
                            Acid_Keyword_Boundary=1,Alkaline_Keyword_Boundary=-0.1,Neutral_Keyword_Boundary=-0.1)
            ###Recording###
            Bug_Note='Normal,only acid.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence=final_HWP_value_single,'NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    ###################################         
    ###Only alkaline：1 and 1+  time###
    ###################################
    elif (len(Acid_index_list)==0)*(len(Alkaline_index_list)>=1)*(len(Neutral_index_list)==0)==1:
        print('Only alkaline, appearance: ',len(Alkaline_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Most time, for single value, choose the maximum if multiple are found### 
            final_HWP_value_single=max(HWP_values_list)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=1,
                            Acid_Keyword_Boundary=-0.1,Alkaline_Keyword_Boundary=1,Neutral_Keyword_Boundary=-0.1)
            ###Recording###
            Bug_Note='Normal,only alkaline.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN',final_HWP_value_single,'NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
            
    ##################################        
    ###Only neutral：1 and 1+  time###
    ##################################
    elif (len(Acid_index_list)==0)*(len(Alkaline_index_list)==0)*(len(Neutral_index_list)>=1)==1:
        print('Only neutral, appearance: ',len(Neutral_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Most time, for single value, choose the maximum if multiple are found### 
            final_HWP_value_single=max(HWP_values_list)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=1,
                            Acid_Keyword_Boundary=-0.1,Alkaline_Keyword_Boundary=-0.1,Neutral_Keyword_Boundary=1)
            ###Recording###
            Bug_Note='Normal,only neutral.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN',final_HWP_value_single,Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    ######################################        
    ###Acid and alkaline：1 and 1+ time###
    ######################################
    elif (len(Acid_index_list)>=1)*(len(Alkaline_index_list)>=1)*(len(Neutral_index_list)==0)==1:
        print('Acid and alkaline, appearance: ',len(Acid_index_list),'and ',len(Alkaline_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=2,
                            Acid_Keyword_Boundary=1,Alkaline_Keyword_Boundary=1,Neutral_Keyword_Boundary=-0.1)
            ###Definition of HWP values according to the sequence###
            if Acid_index_list[-1]<Alkaline_index_list[-1]:
                ###Acid-alkaline###
                final_HWP_value_Acid=HWP_values_list[0]
                final_HWP_value_Alkaline=HWP_values_list[1]
                final_HWP_value_Neutral='NaN'
            ###Alkaline-acid###
            else:
                final_HWP_value_Acid=HWP_values_list[1]
                final_HWP_value_Alkaline=HWP_values_list[0]
                final_HWP_value_Neutral='NaN'
            ###Recording###
            Bug_Note='Normal,acid and alkaline.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence=final_HWP_value_Acid,final_HWP_value_Alkaline,final_HWP_value_Neutral,Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    #####################################        
    ###Acid and neutral：1 and 1+ time###
    #####################################
    elif (len(Acid_index_list)>=1)*(len(Alkaline_index_list)==0)*(len(Neutral_index_list)>=1)==1:
        print('Acid and neutral, appearance: ',len(Acid_index_list),'and ',len(Neutral_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=2,
                            Acid_Keyword_Boundary=1,Alkaline_Keyword_Boundary=-0.1,Neutral_Keyword_Boundary=1)
            ###Definition of HWP values according to the sequence###
            if Acid_index_list[-1]<Neutral_index_list[-1]:
            ###Acid-neutral###
                final_HWP_value_Acid=HWP_values_list[0]
                final_HWP_value_Alkaline='NaN'
                final_HWP_value_Neutral=HWP_values_list[1]
            ###Neutral-acid###
            else:
                final_HWP_value_Acid=HWP_values_list[1]
                final_HWP_value_Alkaline='NaN'
                final_HWP_value_Neutral=HWP_values_list[0]
            ###Recording###
            Bug_Note='Normal,acid and alkaline.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence=final_HWP_value_Acid,final_HWP_value_Alkaline,final_HWP_value_Neutral,Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    #########################################            
    ###Alkaline and neutral：1 and 1+ time###
    #########################################
    elif (len(Acid_index_list)==0)*(len(Alkaline_index_list)>=1)*(len(Neutral_index_list)>=1)==1:
        print('Alkaline and neutral, appearance: ',len(Alkaline_index_list),'and ',len(Neutral_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=2,
                            Acid_Keyword_Boundary=-0.1,Alkaline_Keyword_Boundary=1,Neutral_Keyword_Boundary=1)
            ###Definition of HWP values according to the sequence###
            if Alkaline_index_list[-1]<Neutral_index_list[-1]:
                ###Alkaline-neutral###
                final_HWP_value_Acid='NaN'
                final_HWP_value_Alkaline=HWP_values_list[0]
                final_HWP_value_Neutral=HWP_values_list[1]
            ###Neutral-alkalilne###
            else:
                final_HWP_value_Acid='NaN'
                final_HWP_value_Alkaline=HWP_values_list[1]
                final_HWP_value_Neutral=HWP_values_list[0]
            ###Recording###
            Bug_Note='Normal,acid and alkaline.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence=final_HWP_value_Acid,final_HWP_value_Alkaline,final_HWP_value_Neutral,Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except:
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    
    ###############################################           
    ###Acid, alkaline and neutral：1 and 1+ time###
    ###############################################
    elif (len(Acid_index_list)>=1)*(len(Alkaline_index_list)>=1)*(len(Neutral_index_list)>=1)==1:
        print('Acid, alkaline and neutral, appearance: ',len(Acid_index_list),'and ',len(Alkaline_index_list),'and ',len(Neutral_index_list),'times')
        try:
            founded_values=re.findall(Value_Keyword_pattern,Target_Sentence)
            ###Remove the units###
            HWP_values_list=extract_value(founded_values)
            ###Check Irregular###
            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Check_Irregular(HWP_values_list,
                            Acid_index_list,Alkaline_index_list,
                            Neutral_index_list,Low_Value_Boundary=Low_Value_Boundary,
            High_Value_Boundary=High_Value_Boundary,HWP_Multiple_Boundary=3,
                            Acid_Keyword_Boundary=1,Alkaline_Keyword_Boundary=1,Neutral_Keyword_Boundary=1)
            ###Definition of HWP values according to the sequence###
            ###Acid-alkaline-neutral
            if Acid_index_list[-1]<Alkaline_index_list[-1]<Neutral_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[0]
                final_HWP_value_Alkaline=HWP_values_list[1]
                final_HWP_value_Neutral=HWP_values_list[2]
            ###Acid-neutral-alkaline
            elif Acid_index_list[-1]<Neutral_index_list[-1]<Alkaline_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[0]
                final_HWP_value_Alkaline=HWP_values_list[2]
                final_HWP_value_Neutral=HWP_values_list[1]
            ###Alkaline-acid-neutral
            elif Alkaline_index_list[-1]<Acid_index_list[-1]<Neutral_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[1]
                final_HWP_value_Alkaline=HWP_values_list[0]
                final_HWP_value_Neutral=HWP_values_list[2]
            ###Alkaline-neutral-acid
            elif Alkaline_index_list[-1]<Neutral_index_list[-1]<Acid_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[2]
                final_HWP_value_Alkaline=HWP_values_list[0]
                final_HWP_value_Neutral=HWP_values_list[1]
            ###Neutral-acid-alkaline
            elif Neutral_index_list[-1]<Acid_index_list[-1]<Alkaline_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[1]
                final_HWP_value_Alkaline=HWP_values_list[2]
                final_HWP_value_Neutral=HWP_values_list[0]
            ###Neutral-alkaline-acid
            elif Neutral_index_list[-1]<Alkaline_index_list[-1]<Acid_index_list[-1]:
                final_HWP_value_Acid=HWP_values_list[2]
                final_HWP_value_Alkaline=HWP_values_list[1]
                final_HWP_value_Neutral=HWP_values_list[0]
            ###Recording###
            Bug_Note='Normal,acid, alkaline and neutral.'
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence=final_HWP_value_Acid,final_HWP_value_Alkaline,final_HWP_value_Neutral,Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value
            Irregular_Multiple_Keywords=Irregular_Multiple_Keywords
        except Exception as e:
            print(e)
            print('Error in finding HWP values.',DOI)
            Bug_Note='Error in finding HWP values.'+DOI
            ###Recording###
            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN',Target_Sentence
            Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value=Bug_Note,'NaN','NaN'
            Irregular_Multiple_Keywords='NaN'
    return Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords

In [ ]:
def crawler_by_batch(query_list,Low_Value_Boundary,High_Value_Boundary):
    ###defining a empty dataframe for each batch###
    database=pd.DataFrame(columns=[
    ####publication informations####
    'DOI','Country', 'Facility', 'Journal_Name', 'IF', 'Indexed_Date','Cited_Time',
    ####needed performance output####
    'Acid_HWP','Alkaline_HWP','Neutral_HWP','Remark_Sentence','Bug_Note',
    ####records of irregular informations
    'Irregular_Abnormal_Value','Irregular_Multiple_Value','Irregular_Multiple_Keywords',
    ####elemental informations####
    'Element_1','Element_2','Element_3','Element_4','Element_5','Element_6','Element_7','Element_8',
    ####one-hot features####
    'Judge_Nanoparticles','Judge_3D','Judge_2D','Judge_1D','Judge_Urchin','Judge_SAC','Judge_QD',
    'Judge_Hollow','Judge_Core_Shell','Judge_Template','Judge_alloy','Judge_Functionalized',
    'Judge_heterostructures','Judge_Polymer_Complex_Chelate','Judge_Biomass_Slag','Judge_MOF','Judge_COF',
    'Judge_Conductive_Polymer','Judge_Polymetal','Judge_Hydroxide_Oxyhydroxide','Judge_oxalate','Judge_Spinel',
    'Judge_Perovskite','Judge_Pyrochlore','Judge_Prussian_Blue','Judge_Amorphous','Judge_Ultrasound','Judge_Plasma',
    'Judge_Graphdiyne','Judge_Graphene','Judge_c3n4','Judge_MXene','Judge_CNT','Judge_IL','Judge_Organic_Inorganic',
    'Judge_Steel','Judge_Carbon_Cloth','Judge_Film','Judge_Molten','Judge_Co_Precipitation','Judge_Electrospin',
    'Judge_Hydrothermal','Judge_Microwave','Judge_Spray','Judge_Phys_Chem_Deposition','Judge_Light_Deposition',
    'Judge_Electrochemical_Deposition','Judge_Electrochemical_Corrosion','Judge_Etching','Judge_Sol_Gel',
    ####special warings####
    'Warning_Onset'
    ])
    ####start crawling####
    driver = webdriver.Chrome(options=options)
    i=1
    driver.maximize_window()
    for query_each_DOI in query_list:
        DOI=query_each_DOI[0]
        try:
        ###enter main page
            test=DOI
            try:
                driver.get(url_base)
            except:
                print('link to Webofscience failed')
            if i==1:
                time.sleep(random.uniform(2,3))
            else:
                time.sleep(random.uniform(1.5,2.5))

            ###clear the welcome pages for the first record
            if i==1:
                ###close "coockies" page
                try:
                    driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
                    time.sleep(2)
                except:
                    pass
                ###close "welcome" page
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                    time.sleep(2)
                except:
                    pass
                ###close "tutorial" page
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-4d6d27fa"]').click()
                    time.sleep(2)
                except:
                    pass
                ###close "explore" page
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-586b9e0a"]').click()
                    time.sleep(2)
                except:
                    pass
                ###some times coockies need to be closed again
                try:
                    driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
                    time.sleep(2)
                except:
                    pass
                ###some times welcome need to be closed again
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                    time.sleep(2)
                except:
                    pass
                ###some times tutorial need to be closed again
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-4d6d27fa"]').click()
                    time.sleep(2)
                except:
                    pass
                ###some times explore need to be closed again
                try:
                    driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-586b9e0a"]').click()
                    time.sleep(2)
                except:
                    pass
            else:
                pass
            ###close the survey (before search)
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-button-54e2fb7b"]').click()
                time.sleep(random.uniform(0.4,0.6))
            except:
                pass
            ###some times coockies need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
                time.sleep(2)
            except:
                pass
            ###some times welcome need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                time.sleep(2)
            except:
                pass
            ###some times tutorial need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-4d6d27fa"]').click()
                time.sleep(2)
            except:
                pass
            ###some times explore need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-586b9e0a"]').click()
                time.sleep(2)
            except:
                pass
            ###close the survey (before search 2)
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-button-54e2fb7b"]').click()
                time.sleep(random.uniform(0.4,0.6))
            except:
                pass
            ###some times coockies need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
                time.sleep(2)
            except:
                pass
            ###some times welcome need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                time.sleep(2)
            except:
                pass
            ###some times tutorial need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-4d6d27fa"]').click()
                time.sleep(2)
            except:
                pass
            ###some times explore need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-586b9e0a"]').click()
                time.sleep(2)
            except:
                pass
            ###clear the search colunm
            try:
                driver.find_element(By.XPATH,'//*[@id="snSearchType"]/div[3]/button[1]/span[1]').click()
                print('clear done')
                time.sleep(2)
            except:
                print('clear failed')
            ###some times welcome need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                time.sleep(2)
            except:
                pass
            ###some times welcome need to be closed again
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-5600f670"]').click()
                time.sleep(2)
            except:
                pass

            ###choose to search by DOI
            try:
                time.sleep(random.uniform(1.5,2.5))
                driver.find_element(By.XPATH,'//*[@id="snSearchType"]/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button').click()
                time.sleep(random.uniform(2,3))
                driver.find_element(By.XPATH,'//*[@id="global-select"]/div[1]/div[2]/div[11]').click()
                time.sleep(random.uniform(1.5,2.5))
            except:
                try:
                    driver.refresh()
                    time.sleep(random.uniform(1.5,2.5))
                    driver.find_element(By.XPATH,'//*[@id="snSearchType"]/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button').click()
                    time.sleep(random.uniform(2,3))
                    driver.find_element(By.XPATH,'//*[@id="global-select"]/div[1]/div[2]/div[11]').click()
                    time.sleep(random.uniform(1.5,2.5))
                except:
                    print('choose DOI failed',e)
            ###clear the column,input of DOI number and search
            try:
                if i==1:
                    try:
                        driver.find_element(By.XPATH,'//*[@id="mat-input-0"]').send_keys('%s\n' % test)
                        time.sleep(random.uniform(8.5,9.5))
                        driver.find_element(By.XPATH,'//*[@id="snSearchType"]/div[4]/button[2]/span[1]').click()
                        time.sleep(random.uniform(2.5,3.5))
                    except:
                        print('error')
                else:
                    driver.find_element(By.XPATH,'//*[@id="mat-input-0"]').send_keys('%s\n' % test)
                    time.sleep(random.uniform(8.5,9.5))
                    try:
                        driver.find_element(By.XPATH,'//*[@id="snSearchType"]/div[4]/button[2]/span[1]').click()
                        time.sleep(random.uniform(2.5,3.5))
                    except:
                        pass
            except Exception as e:
                print("search failed")
                print(e)
            ###close the survey (after search)
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-button-54e2fb7b"]').click()
                time.sleep(random.uniform(0.5,0.6))
            except:
                print('no survey need to close')
            ###close the explore (after search)
            try:
                driver.find_element(By.XPATH,'//*[@id="pendo-close-guide-586b9e0a"]').click()
                time.sleep(2)
            except:
                print('close explore skipped')
            ###enter the detailed page
            try:
                driver.find_element(By.XPATH,'/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-records-list/app-record/div/div/div[2]/div[1]/app-summary-title/h3/a').click()
                time.sleep(random.uniform(1.5,2.5))
            except Exception as e:
                print("enter detail page failed")
                print(e)

            ###gather information needed
            try:
                Abstract=driver.find_element(By.XPATH,'//*[@id="FullRTa-abstract-basic"]/p').text
            except:
                Abstract='NaN'

            ####gather information of material and elements###
            try:
                ###Publishing related informations###
                try:
                    Indexed_Date=driver.find_element(By.XPATH,'//*[@id="FullRTa-indexedDate"]').text
                except:
                    try:
                        time.sleep(2)
                        Indexed_Date=driver.find_element(By.XPATH,'//*[@id="FullRTa-indexedDate"]').text
                    except:
                        Indexed_Date='NaN'
                try:
                    Cited_Time=driver.find_element(By.XPATH,'//*[@id="FullRRPTa-citationsLabelPlural-ALLDB"]').text
                    Cited_Time=int(Cited_Time)
                except:
                    try:
                        time.sleep(2)
                        Cited_Time=driver.find_element(By.XPATH,'//*[@id="snCitationData"]/div[1]/div[1]/div/div').text
                        Cited_Time=int(Cited_Time)
                    except:
                        Cited_Time=0
                try:
                    IF=driver.find_element(By.XPATH,'//*[@id="snJournalData"]/div[2]/div[1]/div[1]/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/button/span[1]/span').text
                    IF=float(IF)
                except:
                    try:
                        time.sleep(2)
                        IF=driver.find_element(By.XPATH,'//*[@id="snJournalData"]/div[2]/div[1]/div[1]/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/button/span[1]/span').text
                        IF=float(IF)
                    except:
                        IF=0
                try:
                    Journal_Name=driver.find_element(By.XPATH,'//*[@id="snJournalData"]/div[1]/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/a').text
                except:
                    try:
                        time.sleep(2)
                        Journal_Name=driver.find_element(By.XPATH,'//*[@id="snJournalData"]/div[1]/app-jcr-sidenav/mat-sidenav-container/mat-sidenav-content/span/a').text
                    except:
                        Journal_Name='NaN'
                try:
                    address=driver.find_element(By.XPATH,'//*[@id="AiinTa-RepAddressFull-0"]').text
                except:
                    try:
                        time.sleep(2)
                        address=driver.find_element(By.XPATH,'//*[@id="AiinTa-RepAddressFull-0"]').text
                    except:
                        address='NaN'
                try:
                    Facility=address.split(', ')[0]
                    Country=address.split(', ')[-1]
                except:
                    Facility='NaN'
                    Country='NaN'
                ####gather information of material and elements###
                try:
                    HWP_found_sentence_list=Find_Target_Sentence_From_Text(Abstract,Keyword_pattern=Keyword_pattern_HWP)
                    Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset=Extract_target_element_material(Not_needed_key_word_list,text=Abstract)
                    ###Whether no key word found###
                    if len(HWP_found_sentence_list)==0:
                        print('No HWP keyword found',DOI)
                        Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence='NaN','NaN','NaN','NaN'
                        Bug_Note='Error in finding HWP keyword.'+DOI
                        Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords='NaN','NaN','NaN'
                        ###
                        database=Add_New_Record(
                        DOI,Country,Facility,Journal_Name,IF,Indexed_Date,Cited_Time,
                        Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,
                        Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords,
                        Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,
                        Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,
                        Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,
                        Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,
                        Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,
                        Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,
                        Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,
                        Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,
                        Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,
                        Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset,
                        database)
                    ###Whether single sentence###
                    elif len(HWP_found_sentence_list)==1:
                        Target_Sentence=HWP_found_sentence_list[0]
                        print("Single sentence found")
                        Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Extracting_target_values(Acid_pattern,
                        Alkaline_pattern,
                        Neutral_pattern,
                        Value_Keyword_pattern,
                        Target_Sentence,
                        Low_Value_Boundary,High_Value_Boundary,DOI
                        )
                        ###
                        database=Add_New_Record(
                        DOI,Country,Facility,Journal_Name,IF,Indexed_Date,Cited_Time,
                        Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,
                        Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords,
                        Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,
                        Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,
                        Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,
                        Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,
                        Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,
                        Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,
                        Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,
                        Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,
                        Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,
                        Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset,
                        database)
                    elif len(HWP_found_sentence_list)>1:
                        print("Multiple sentence found")
                        for i in range(0,len(HWP_found_sentence_list)):
                            Target_Sentence=HWP_found_sentence_list[i]
                            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords=Extracting_target_values(Acid_pattern,
                            Alkaline_pattern,
                            Neutral_pattern,
                            Value_Keyword_pattern,
                            Target_Sentence,
                            Low_Value_Boundary,High_Value_Boundary,DOI
                            )
                            ###
                            database=Add_New_Record(
                            DOI,Country,Facility,Journal_Name,IF,Indexed_Date,Cited_Time,
                            Acid_HWP,Alkaline_HWP,Neutral_HWP,Remark_Sentence,Bug_Note,
                            Irregular_Abnormal_Value,Irregular_Multiple_Value,Irregular_Multiple_Keywords,
                            Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,
                            Judge_Nanoparticles,Judge_3D,Judge_2D,Judge_1D,Judge_Urchin,Judge_SAC,Judge_QD,
                            Judge_Hollow,Judge_Core_Shell,Judge_Template,Judge_alloy,Judge_Functionalized,
                            Judge_heterostructures,Judge_Polymer_Complex_Chelate,Judge_Biomass_Slag,Judge_MOF,Judge_COF,
                            Judge_Conductive_Polymer,Judge_Polymetal,Judge_Hydroxide_Oxyhydroxide,Judge_oxalate,Judge_Spinel,
                            Judge_Perovskite,Judge_Pyrochlore,Judge_Prussian_Blue,Judge_Amorphous,Judge_Ultrasound,Judge_Plasma,
                            Judge_Graphdiyne,Judge_Graphene,Judge_c3n4,Judge_MXene,Judge_CNT,Judge_IL,Judge_Organic_Inorganic,
                            Judge_Steel,Judge_Carbon_Cloth,Judge_Film,Judge_Molten,Judge_Co_Precipitation,Judge_Electrospin,
                            Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,
                            Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel,Warning_Onset,
                            database)
                except Exception as e:
                    print(e)
                    print(traceback.print_exc())
                    print('Parsing material information failed',DOI)
            except:
                print("gather information failed")
        except:
            print('initial step error.')
        i+=1
        print(i)
        if(i%10==1):
            time.sleep(random.uniform(10,20))
    driver.close()
    return database

In [ ]:
time_begin=time.time()
for each_sub_batch in splitted:
    database_batch=crawler_by_batch(each_sub_batch,Low_Value_Boundary=0.5,High_Value_Boundary=1)
    #update the database
    database=database.append(database_batch)
time_end=time.time()
cost_time=(time_end-time_begin)//3600
print('Cost time: %d h'%cost_time)

In [ ]:
database.to_csv("results_ORR1.csv",encoding='utf-8')